In [1]:
import pandas as pd

In [2]:
bike_realtime_df = pd.read_csv('./bike_realtime0416-0519.csv')
mrt_history_df = pd.read_csv('./mrt_history_groupby03-04.csv')
time_table_df = pd.read_csv('./time_table.csv')
youbike_mrt_distance_df = pd.read_csv('./youbike_mrt_distance.csv')
bike_station_df = pd.read_csv('./bike_station.csv')
mrt_station_info_df = pd.read_csv('./mrt_station_info.csv')
bike_realtime_df.drop(columns="create_time",inplace=True)

In [3]:
bike_realtime_df["source_time"] = pd.to_datetime(bike_realtime_df["source_time"]) 
bike_realtime_df["month"] = bike_realtime_df["source_time"].dt.month
bike_realtime_df["date"] = bike_realtime_df["source_time"].dt.date
bike_realtime_df["hour"] = bike_realtime_df["source_time"].dt.hour
bike_realtime_df["mins"] = bike_realtime_df["source_time"].dt.minute

In [4]:
bike_realtime_df["date"] = pd.to_datetime(bike_realtime_df["date"])
time_table_df["date"] = pd.to_datetime(time_table_df["date"])
mrt_history_df["date"] = pd.to_datetime(mrt_history_df["date"])

In [12]:
start_date = '2024-04-16'
end_date = '2024-04-30'

mrt_history_df = mrt_history_df[(mrt_history_df["date"] >= start_date) & (mrt_history_df["date"] <= end_date)]

In [5]:
aval_bike_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_bike"].mean()).reset_index(drop=False)
aval_bike_mean.rename(columns={"aval_bike":"aval_bike_mean"},inplace=True)
aval_space_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_space"].mean()).reset_index(drop=False)
aval_space_mean.rename(columns={"aval_space":"aval_space_mean"},inplace=True)
df_bike_hour_grby = aval_bike_mean.merge(aval_space_mean,
                     how="left",
                     left_on=["bike_station_id","date","hour"],
                     right_on=["bike_station_id","date","hour"])

In [8]:
youbike_mrt_distance_df = youbike_mrt_distance_df.loc[youbike_mrt_distance_df["distance"]<=0.5,]

In [9]:
mrt_station_table = youbike_mrt_distance_df.merge(mrt_station_info_df.loc[:,["mrt_station_id","station_name"]],
                                 on="mrt_station_id",
                                 how="left")
mrt_station_table

,bike_station_id,mrt_station_id,distance,station_name
0,500107153,BR15,0.352221,劍南路
1,500107132,BR15,0.072653,劍南路
2,500107133,BR15,0.242977,劍南路
3,500107008,BR15,0.093802,劍南路
4,500108121,BR15,0.457363,劍南路
...,...,...,...,...
966,500113068,BL10,0.023105,龍山寺
967,500113043,BL10,0.326785,龍山寺
968,500113023,BL10,0.401119,龍山寺
969,500113042,BL10,0.390115,龍山寺


In [15]:
mrt_history_df

,date,hour,mrt_station,enter_num,exit_num
113988,2024-04-16,0,龍山寺,127,142.0
113989,2024-04-16,0,麟光,10,30.0
113990,2024-04-16,0,頭前庄,15,35.0
113991,2024-04-16,0,頂溪,71,230.0
113992,2024-04-16,0,頂埔,8,104.0
...,...,...,...,...,...
151153,2024-04-30,23,三重國小,116,423.0
151154,2024-04-30,23,三重,87,183.0
151155,2024-04-30,23,三民高中,89,453.0
151156,2024-04-30,23,三和國中,99,356.0


In [18]:
mrt_cover_range_table_df = mrt_station_table.merge(mrt_history_df,left_on=["station_name"],right_on=["mrt_station"],how="left")
mrt_cover_range_table_df

,bike_station_id,mrt_station_id,distance,station_name,date,hour,mrt_station,enter_num,exit_num
0,500107153,BR15,0.352221,劍南路,2024-04-16,0.0,劍南路,79.0,31.0
1,500107153,BR15,0.352221,劍南路,2024-04-16,1.0,劍南路,0.0,0.0
2,500107153,BR15,0.352221,劍南路,2024-04-16,5.0,劍南路,0.0,0.0
3,500107153,BR15,0.352221,劍南路,2024-04-16,6.0,劍南路,103.0,190.0
4,500107153,BR15,0.352221,劍南路,2024-04-16,7.0,劍南路,561.0,862.0
...,...,...,...,...,...,...,...,...,...
301150,500113044,BL10,0.344044,龍山寺,2024-04-30,19.0,龍山寺,1418.0,2322.0
301151,500113044,BL10,0.344044,龍山寺,2024-04-30,20.0,龍山寺,1080.0,1554.0
301152,500113044,BL10,0.344044,龍山寺,2024-04-30,21.0,龍山寺,1046.0,1533.0
301153,500113044,BL10,0.344044,龍山寺,2024-04-30,22.0,龍山寺,765.0,1338.0


In [1]:
pd.DataFrame(mrt_cover_range_table_df.groupby(["bike_station_id","date","hour"])["enter_num"].sum()).reset_index(drop=False)

NameError: name 'pd' is not defined